# Agent CFO — Performance Optimization & Design

---
This is Group 8's notebook for the ICT3113 Performance Testing and Optimization project following the required structure.


You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.


## 1. Config & Secrets

Fill in your API keys in secrets. **Do not hardcode keys** in cells.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

COMPANY_NAME = "NVIDIA"

## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

In [23]:
# Install dependencies for ingestion pipeline
%pip install langchain-community langchain-openai faiss-cpu rank-bm25 openai tiktoken pypdf pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install pyinstrument

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import libraries for ingestion pipeline

In [2]:
import time
import glob
import json
import re
import pdfplumber
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.retrievers import SVMRetriever
from langchain_core.retrievers import BaseRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from typing import List, Dict, Optional
import os

c:\Users\cryst\VSCode\repos\pto-rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF Parser and Chunking (Using pdfplumber)
1. Extracts words from pdf
2. Detects sections
3. Returns langchain Document object with page content and metadata containing page number, report name, section, and type of content

In [3]:
def detect_section(page, text, min_size_diff=2.0):
    """
    Try to extract the section title from page text using regex patterns or font size, weight.
    Handles 10-K, 10Q, Presentations, Investor PDFs and similar formats.
    """
    patterns = [
        r"(Item\s+\d+[A-Za-z]?.\s*[A-Z][^\n]+)",  # e.g. "Item 7. Management’s Discussion..."
        r"(ITEM\s+\d+[A-Z]?.\s*[A-Z][^\n]+)",     # uppercase variant
        r"(^[A-Z][A-Z\s]{10,})"                   # fallback for all-caps section titles
    ]
    for p in patterns:
        m = re.search(p, text)
        if m:
            return m.group(1).strip()

    # For presentation decks/investor pdfs
    try:
        words = page.extract_words(extra_attrs=["size", "fontname"])
        if not words:
            return None

        from collections import defaultdict
        lines = defaultdict(list)

        # Group words by y-position
        for w in words:
            lines[round(w["top"], -1)].append(w)

        # Compute global average font size to compare against
        avg_font_size = sum(float(w["size"]) for w in words) / len(words)

        # Define line scoring function
        def score_line(line):
            avg_size = sum(float(w["size"]) for w in line) / len(line)
            bold_bonus = any("Bold" in w["fontname"] or "Black" in w["fontname"] for w in line)
            return avg_size + (3 if bold_bonus else 0), avg_size

        # Score all lines
        scored_lines = [(y, line, *score_line(line)) for y, line in lines.items()]
        if not scored_lines:
            return None

        # Pick the top scoring line
        _, best_line, best_score, best_size = max(scored_lines, key=lambda x: x[2])

        # If the line isn't significantly larger or bold, discard it
        if best_size < avg_font_size + min_size_diff:
            # (e.g. all text is 10pt, best line is 11pt → not a section)
            return None

        # Skip lines that look too long (to avoid full sentences)
        if len(best_line) > 20:
            return None

        section_title = " ".join(w["text"] for w in best_line)
        return section_title.strip() if section_title else None

    except Exception:
        return None

    return None


def parse_pdf_with_tables(pdf_path, report_name, year=None):
    text_docs, table_docs = [], []
    current_section = None  # rolling context

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            tables = page.extract_tables() or []

            # --- Try to detect section header ---
            section = detect_section(page, text)
            if section:
                current_section = section  # update rolling section

            # --- Create text Document ---
            text_docs.append(
                Document(
                    page_content=text,
                    metadata={
                        "page": i,
                        "report": report_name,
                        "year": year,
                        "section": current_section,
                        "type": "text"
                    }
                )
            )

            # --- Create table Documents ---
            for t in tables:
                cleaned_table = [
                    [cell if cell is not None else "" for cell in row]
                    for row in t
                ]
                table_text = "\n".join(["\t".join(row) for row in cleaned_table])
                table_docs.append(
                    Document(
                        page_content=table_text,
                        metadata={
                            "page": i,
                            "report": report_name,
                            "year": year,
                            "section": current_section,
                            "type": "table"
                        }
                    )
                )

    return text_docs, table_docs

Main ingestion pipeline

1.   Loops over all pdfs in folder
2.   Checks cache for parsed pdf
3.   Load from cache if exists, avoiding re-parsing pdf
4.   Save to cache after parsing
5.   Use parallel processing to parse pdfs in parallel



In [5]:
# --- Parameters ---
pdf_folder = "data/"

PARSED_DIR = "cache/parsed_pdfs"
os.makedirs(PARSED_DIR, exist_ok=True)

def parse_pdf_cached(pdf_path, report_name, year=None):
    cache_file = os.path.join(PARSED_DIR, f"{report_name}.json")

    try:
        # --- If cached and file not modified, skip re-parsing ---
        # pdf_mtime = os.path.getmtime(pdf_path)
        # if os.path.exists(cache_file):
        #     # cache_mtime = os.path.getmtime(cache_file)
        #     # if cache_mtime > pdf_mtime:
        #     print(f"Using cached parse for {report_name}\n")
        #     with open(cache_file, "r", encoding="utf-8") as f:
        #         data = json.load(f)
        #     docs = [Document(**d) for d in data]
        #     return docs

        # --- Otherwise, parse fresh ---
        print(f"Parsing {os.path.basename(pdf_path)} ...\n")
        text_docs, table_docs = parse_pdf_with_tables(pdf_path, report_name, year)
        docs = text_docs + table_docs

        # Save to cache
        serializable_docs = [
            {"page_content": d.page_content, "metadata": d.metadata}
            for d in docs
        ]
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(serializable_docs, f, ensure_ascii=False, indent=2)

        return docs
    except Exception as e:
        print(f"Error parsing PDF {report_name}: {e}")
        return []


def process_pdf(pdf_path):
    report_name = os.path.basename(pdf_path).replace(".pdf", "")
    return parse_pdf_cached(pdf_path, report_name)


# Configuration for benchmark runs
NUM_RUNS = 10
RESULTS_DIR = "baseline_ingestion_results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# Store all runs timing data
all_ingestion_runs = []

for run_num in range(1, NUM_RUNS + 1):
    print(f"\n{'#'*70}")
    print(f"# INGESTION RUN {run_num}/{NUM_RUNS}")
    print(f"{'#'*70}\n")
    
    # --- Measure time!! ----
    start_total = time.perf_counter()
    all_docs = []

    for pdf_path in glob.glob(os.path.join(pdf_folder, "*.pdf")):
        doc = process_pdf(pdf_path)
        all_docs.extend(doc)

    # with ThreadPoolExecutor(max_workers=4) as executor:
    #     results = list(executor.map(process_pdf, glob.glob(os.path.join(pdf_folder, "*.pdf"))))
    #     for docs in results:
    #         all_docs.extend(docs)

    elapsed_total = time.perf_counter() - start_total
    
    # Store run results
    run_data = {
        "run_number": run_num,
        "T_ingestion": elapsed_total,
        "num_documents": len(all_docs),
        "num_pdfs": len(glob.glob(os.path.join(pdf_folder, "*.pdf")))
    }
    all_ingestion_runs.append(run_data)
    
    print(f"[Run {run_num}] Loaded {len(all_docs)} parsed documents in {elapsed_total:.2f}s.")

    # Save all ingestion timing results
    ingestion_results_file = f"{RESULTS_DIR}/baseline_ingestion_timing.json"
    with open(ingestion_results_file, 'w', encoding='utf-8') as f:
        json.dump(all_ingestion_runs, f, indent=2, ensure_ascii=False)

# Calculate statistics
import statistics
ingestion_times = [run['T_ingestion'] for run in all_ingestion_runs]

print(f"\n{'#'*70}")
print(f"# INGESTION BENCHMARK COMPLETE - {NUM_RUNS} RUNS")
print(f"{'#'*70}\n")
print(f"Results saved to: {ingestion_results_file}\n")

print(f"Ingestion time statistics:")
print(f"  Mean: {statistics.mean(ingestion_times):.4f}s")
print(f"  Median: {statistics.median(ingestion_times):.4f}s")
print(f"  Std Dev: {statistics.stdev(ingestion_times):.4f}s")
print(f"  Min: {min(ingestion_times):.4f}s")
print(f"  Max: {max(ingestion_times):.4f}s")


######################################################################
# INGESTION RUN 1/10
######################################################################

Parsing FY21_10K.pdf ...

Parsing FY22_10K.pdf ...

Parsing FY23_10K.pdf ...

Parsing FY24Q1_10Q.pdf ...

Parsing FY24Q2_10Q.pdf ...

Parsing FY24Q3_10Q.pdf ...



Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern4' is an invalid float value
Cannot set gray non-stroke color because /'Pattern5' is an invalid float value
Cannot set gray non-stroke color because /'Pattern6' is an invalid float value
Cannot set gray non-stroke color because /'Pattern7' is an invalid float value
Cannot set gray non-stroke color because /'Pattern8' is an invalid float value
Cannot set gray non-stroke color because /'Pattern9' is an invalid float value
Cannot set gray non-stroke color because /'Pattern10' is an invalid float value
Cannot set gray non-stroke color because /'Pattern11' is an invalid float value
Cannot set gray non-stroke color because /'Pattern12' is an invalid float value
Cannot set gray non-stroke color because /'Patter

Parsing FY24_10K.pdf ...

Parsing FY25Q1_10Q.pdf ...

Parsing FY25Q2_10Q.pdf ...



Cannot set gray non-stroke color because /'P25' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value


Parsing FY25Q2_QuarterlyPresentation.pdf ...



...

Parsing FY26Q2_PR.pdf ...



Cannot set gray non-stroke color because /'P25' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value


Parsing FY26Q2_QuarterlyPresentation.pdf ...



Cannot set gray non-stroke color because /'P129' is an invalid float value
Cannot set gray non-stroke color because /'P135' is an invalid float value
Cannot set gray non-stroke color because /'P143' is an invalid float value
Cannot set gray non-stroke color because /'P227' is an invalid float value


[Run 10] Loaded 2039 parsed documents in 1107.62s.

######################################################################
# INGESTION BENCHMARK COMPLETE - 10 RUNS
######################################################################

Results saved to: baseline_ingestion_results/baseline_ingestion_timing.json

Ingestion time statistics:
  Mean: 861.4824s
  Median: 793.3103s
  Std Dev: 260.4376s
  Min: 568.6176s
  Max: 1191.1444s


In [6]:
all_ingestion_runs

[{'run_number': 1,
  'T_ingestion': 568.6176294999896,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 2,
  'T_ingestion': 617.0319073999999,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 3,
  'T_ingestion': 766.9567005000135,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 4,
  'T_ingestion': 819.6638425999845,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 5,
  'T_ingestion': 1191.144428300002,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 6,
  'T_ingestion': 1165.4266896999907,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 7,
  'T_ingestion': 1137.6328883000242,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 8,
  'T_ingestion': 625.8353982999979,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 9,
  'T_ingestion': 614.8919722999854,
  'num_documents': 2039,
  'num_pdfs': 21},
 {'run_number': 10,
  'T_ingestion': 1107.6228564000048,
  'num_documents': 2039,
  'num_pdfs': 21}]

Chunking
1. Uses langchain's Recursive Character Text splitter to split documents
2. Returns chunked documents

In [28]:
# Chunking parameters
max_chunk_size = 800
chunk_overlap = 100

def chunk_documents(all_docs, chunk_size=max_chunk_size, chunk_overlap=chunk_overlap):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(all_docs)

# Store chunking timing data
all_chunking_runs = []

for run_num in range(1, NUM_RUNS + 1):
    print(f"\n{'#'*70}")
    print(f"# CHUNKING RUN {run_num}/{NUM_RUNS}")
    print(f"{'#'*70}\n")
    
    start_chunking = time.perf_counter()
    chunked_docs = chunk_documents(all_docs, max_chunk_size, chunk_overlap)
    elapsed_chunking = time.perf_counter() - start_chunking
    
    # Store chunking results
    chunking_data = {
        "run_number": run_num,
        "T_chunking": elapsed_chunking,
        "num_chunks": len(chunked_docs),
        "num_documents": len(all_docs)
    }
    all_chunking_runs.append(chunking_data)
    
    # Add to all_ingestion_runs
    all_ingestion_runs[run_num - 1]["T_chunking"] = elapsed_chunking
    all_ingestion_runs[run_num - 1]["num_chunks"] = len(chunked_docs)
    
    print(f"[Run {run_num}] Chunked into {len(chunked_docs)} segments in {elapsed_chunking:.2f}s")
    
    # Save updated results
    with open(f"{RESULTS_DIR}/baseline_ingestion_timing.json", 'w', encoding='utf-8') as f:
        json.dump(all_ingestion_runs, f, indent=2, ensure_ascii=False)

# Calculate chunking statistics
chunking_times = [run['T_chunking'] for run in all_chunking_runs]

print(f"\n{'#'*70}")
print(f"# CHUNKING BENCHMARK COMPLETE - {NUM_RUNS} RUNS")
print(f"{'#'*70}\n")

print(f"Chunking time statistics:")
print(f"  Mean: {statistics.mean(chunking_times):.4f}s")
print(f"  Median: {statistics.median(chunking_times):.4f}s")
print(f"  Std Dev: {statistics.stdev(chunking_times):.4f}s")
print(f"  Min: {min(chunking_times):.4f}s")
print(f"  Max: {max(chunking_times):.4f}s")

print(f"\nParsed and chunked {len(all_docs)} documents from {len(glob.glob(pdf_folder + '*.pdf'))} PDFs.")



######################################################################
# CHUNKING RUN 1/10
######################################################################

[Run 1] Chunked into 6354 segments in 0.69s

######################################################################
# CHUNKING RUN 2/10
######################################################################

[Run 1] Chunked into 6354 segments in 0.69s

######################################################################
# CHUNKING RUN 2/10
######################################################################

[Run 2] Chunked into 6354 segments in 0.29s

######################################################################
# CHUNKING RUN 3/10
######################################################################

[Run 2] Chunked into 6354 segments in 0.29s

######################################################################
# CHUNKING RUN 3/10
######################################################################

[Run

In [29]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small") # Can be 'text-embedding-3-large'

def build_faiss_index(chunked_docs, embedding_model):
    # FAISS Vector Index
    faiss_store = FAISS.from_documents(chunked_docs, embedding_model)
    faiss_store.save_local("faiss_index")
    
    # Load FAISS Vector Store
    faiss_store = FAISS.load_local(
        "faiss_index",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
    return faiss_store

# Store FAISS building timing data
all_faiss_runs = []

for run_num in range(1, NUM_RUNS + 1):
    print(f"\n{'#'*70}")
    print(f"# FAISS BUILDING RUN {run_num}/{NUM_RUNS}")
    print(f"{'#'*70}\n")
    
    start_faiss = time.perf_counter()
    faiss_store = build_faiss_index(chunked_docs, embedding_model)
    elapsed_faiss = time.perf_counter() - start_faiss
    
    # Store FAISS results
    faiss_data = {
        "run_number": run_num,
        "T_faiss_building": elapsed_faiss,
        "num_chunks": len(chunked_docs)
    }
    all_faiss_runs.append(faiss_data)
    
    # Add to all_ingestion_runs
    all_ingestion_runs[run_num - 1]["T_faiss_building"] = elapsed_faiss
    
    # Calculate total time
    total_time = (all_ingestion_runs[run_num - 1]["T_ingestion"] + 
                  all_ingestion_runs[run_num - 1]["T_chunking"] + 
                  elapsed_faiss)
    all_ingestion_runs[run_num - 1]["T_total"] = total_time
    
    print(f"[Run {run_num}] Built FAISS index in {elapsed_faiss:.2f}s")
    print(f"[Run {run_num}] Total pipeline time: {total_time:.2f}s")
    
    # Save updated results
    with open(f"{RESULTS_DIR}/baseline_ingestion_timing.json", 'w', encoding='utf-8') as f:
        json.dump(all_ingestion_runs, f, indent=2, ensure_ascii=False)

# Calculate FAISS building statistics
faiss_times = [run['T_faiss_building'] for run in all_faiss_runs]

print(f"\n{'#'*70}")
print(f"# FAISS BUILDING BENCHMARK COMPLETE - {NUM_RUNS} RUNS")
print(f"{'#'*70}\n")

print(f"FAISS building time statistics:")
print(f"  Mean: {statistics.mean(faiss_times):.4f}s")
print(f"  Median: {statistics.median(faiss_times):.4f}s")
print(f"  Std Dev: {statistics.stdev(faiss_times):.4f}s")
print(f"  Min: {min(faiss_times):.4f}s")
print(f"  Max: {max(faiss_times):.4f}s")

# Display total pipeline statistics
total_times = [run['T_total'] for run in all_ingestion_runs]
print(f"\nTotal pipeline time statistics:")
print(f"  Mean: {statistics.mean(total_times):.4f}s")
print(f"  Median: {statistics.median(total_times):.4f}s")
print(f"  Std Dev: {statistics.stdev(total_times):.4f}s")
print(f"  Min: {min(total_times):.4f}s")
print(f"  Max: {max(total_times):.4f}s")



######################################################################
# FAISS BUILDING RUN 1/10
######################################################################

[Run 1] Built FAISS index in 40.29s
[Run 1] Total pipeline time: 609.60s

######################################################################
# FAISS BUILDING RUN 2/10
######################################################################

[Run 1] Built FAISS index in 40.29s
[Run 1] Total pipeline time: 609.60s

######################################################################
# FAISS BUILDING RUN 2/10
######################################################################

[Run 2] Built FAISS index in 37.83s
[Run 2] Total pipeline time: 655.15s

######################################################################
# FAISS BUILDING RUN 3/10
######################################################################

[Run 2] Built FAISS index in 37.83s
[Run 2] Total pipeline time: 655.15s

###########################